In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/jadefok/113_1/AIIM/final_project/Diabetes data set/base.csv')

### Add label and remove HBA1C_b, FBG_B, DIABETES_SELF_b

In [3]:
df['target'] = np.where((df['HBA1C_b'] >= 6.5) | (df['FBG_B'] >= 126)| (df['DIABETES_SELF_b'] == 1), 1, 0)

In [4]:
df.drop('HBA1C_b', axis=1, inplace=True)
df.drop('FBG_B', axis=1, inplace=True)
df.drop('DIABETES_SELF_b', axis=1, inplace=True)

### Remove columns for follow up (out of scope)

In [5]:
df.drop('DIABETES_SELF', axis=1, inplace=True)
df.drop('basedate', axis=1, inplace=True)
df.drop('followdate', axis=1, inplace=True)
df.drop('HBA1C', axis=1, inplace=True)
df.drop('FBG', axis=1, inplace=True)
df.drop('cardio_f', axis=1, inplace=True)

### Remove ID

In [6]:
df.drop('CaseNo', axis=1, inplace=True)

### Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

### Find outlier

In [9]:
categorical_features = ['SEX', 'edu', 'DM_FAM', 'smoking', 'DRK', 'betel', 'SPORT', 'cardio_b']
continuous_features = ['AGE', 'SBP', 'DBP', 'HR', 'Weight', 'Height', 'BMI', 'WHR', 'T_CHO', 'TG', 'HDL', 'LDL']

##### z_score

The Z-score calculates how far a data point is from the mean, measured in standard deviations. If a value’s Z-score is beyond a certain threshold (e.g., 3), it's flagged as an outlier. This method assumes data is normally distributed, making it suitable for symmetric, bell-shaped distributions.

In [10]:
for feature in continuous_features:
    z_scores = np.abs((train_df[feature] - train_df[feature].mean()) / train_df[feature].std())
    z_score_filtered = train_df[z_scores < 3]

#### Interquartile Range (IQR) Method

IQR measures the spread of the middle 50% of the data (between the first quartile 
𝑄
1
Q1 and third quartile 
𝑄
3
Q3). Values outside 
𝑄
1
−
1.5
×
IQR
Q1−1.5×IQR or 
𝑄
3
+
1.5
×
IQR
Q3+1.5×IQR are considered outliers. IQR is less sensitive to extreme values and works well with skewed data.

In [11]:
for feature in continuous_features:
    Q1, Q3 = np.percentile(train_df[feature] , [25, 75])
    IQR = Q3 - Q1
    IQR_filtered_data = train_df[(train_df[feature]  >= Q1 - 1.5 * IQR) & (train_df[feature]  <= Q3 + 1.5 * IQR)]

#### Median Absolute Deviation (MAD)

MAD is a robust method based on the median and measures the absolute deviation of each value from the median. Dividing by MAD (or scaled by 1.482 for a normal approximation) helps identify outliers by setting a threshold (e.g., 3 MAD). MAD is more robust than Z-score for non-normal distributions and is less influenced by extreme values.

In [12]:
for feature in continuous_features:
    median = np.median(train_df[feature])
    mad = np.median(np.abs(train_df[feature] - median))
    MAD_filtered_data = train_df[np.abs(train_df[feature] - median) / mad < 3]

#### filtered data

In [13]:
overlapping_indices = z_score_filtered.index.intersection(MAD_filtered_data.index).intersection(IQR_filtered_data.index)

In [14]:
filtered_train_df = z_score_filtered.loc[overlapping_indices]

### Imputation

In [15]:
fillna = {}

train_df_columns = filtered_train_df.columns.to_list()

for i in range(0,len(train_df_columns)):
  if train_df_columns[i] in categorical_features:
    fillna[train_df_columns[i]] = filtered_train_df[train_df_columns[i]].mode().values[0]
  if train_df_columns[i] in continuous_features:
    fillna[train_df_columns[i]] = filtered_train_df[train_df_columns[i]].mean()

In [16]:
for index, row in train_df.iterrows():
  for j in train_df:
      if pd.isna(row[j]):
        train_df.loc[index, j] = fillna[j]

In [17]:
for index, row in test_df.iterrows():
  for j in test_df:
      if pd.isna(row[j]):
        test_df.loc[index, j] = fillna[j]

In [18]:
train_df.isnull().sum()

SEX         0
AGE         0
edu         0
DM_FAM      0
SBP         0
DBP         0
HR          0
Weight      0
Height      0
BMI         0
WHR         0
T_CHO       0
TG          0
HDL         0
LDL         0
smoking     0
DRK         0
betel       0
SPORT       0
cardio_b    0
target      0
dtype: int64

### Normalization

In [19]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [20]:
scaler = StandardScaler()
train_df[continuous_features] = scaler.fit_transform(train_df[continuous_features])
test_df[continuous_features] = scaler.fit_transform(test_df[continuous_features])

In [21]:
train_df

,SEX,AGE,edu,DM_FAM,SBP,DBP,HR,Weight,Height,BMI,...,T_CHO,TG,HDL,LDL,smoking,DRK,betel,SPORT,cardio_b,target
2204,2,-1.170684,1.0,0,-1.078911,-0.480133,-0.307474,-1.229788,-0.711959,-1.166503,...,0.269388,-0.730558,1.431123,-0.232979,0,0,0,0,0,0
940,1,0.183639,1.0,0,-0.444249,-0.302312,0.245755,-0.073090,0.342683,-0.293103,...,-1.162910,-0.126866,-0.635494,-1.037786,1,0,0,1,0,0
1815,2,1.176810,2.0,0,2.584821,1.698181,0.467047,-0.049643,-1.297872,0.963707,...,0.870954,-0.291509,-0.045032,1.190911,0,0,0,0,0,0
219,1,-0.177514,1.0,0,-0.444249,0.586796,-0.639412,-0.510760,0.459866,-0.911518,...,1.071475,0.284743,0.840661,0.912324,1,0,0,0,0,0
445,2,0.635081,2.0,1,-0.617339,-0.480133,0.356401,-0.971876,-0.969761,-0.637967,...,-0.446761,-0.291509,-0.118840,-0.294887,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,1,0.635081,2.0,0,0.651986,0.364519,0.135109,0.599045,0.811413,0.216082,...,-0.303531,-0.172600,-0.561687,0.076562,1,0,0,1,0,0
1095,1,0.454504,1.0,0,-0.501946,-2.258349,-0.307474,-0.471682,0.752822,-1.032316,...,-1.993644,-0.017103,-1.373572,-1.533052,1,0,0,0,1,1
1130,2,0.183639,2.0,0,1.257800,0.053331,-1.524579,-0.995322,-0.126047,-1.190581,...,0.498556,-0.648236,0.840661,0.571829,0,0,0,1,1,0
1294,1,-0.990108,2.0,1,0.709682,2.187191,0.798985,0.497443,-0.243229,0.867790,...,-0.389469,-0.227481,0.028776,-0.294887,0,0,0,0,0,0


In [22]:
test_df

,SEX,AGE,edu,DM_FAM,SBP,DBP,HR,Weight,Height,BMI,...,T_CHO,TG,HDL,LDL,smoking,DRK,betel,SPORT,cardio_b,target
2034,2,0.921987,2.0,1,-0.974273,-1.056280,1.058227,-0.797885,-1.585068,0.005378,...,-0.384816,-0.030147,0.592351,-0.663270,0,0,0,1,0,1
1465,2,-0.526510,1.0,0,-0.381806,0.514672,1.173416,-0.960467,-1.005276,-0.650901,...,-1.026008,-0.598003,0.872976,-0.906789,0,0,0,0,0,0
1706,1,0.469332,2.0,1,-1.171762,-1.579931,-2.973373,0.155437,0.966016,-0.387258,...,-1.137520,-0.490860,-0.179367,-1.332948,1,0,0,1,1,0
1710,2,0.831456,0.0,1,0.092168,0.699489,0.712661,0.251508,-0.715380,0.927608,...,-0.579962,0.548422,-0.600304,-0.571950,0,0,0,1,0,0
1259,2,0.288270,2.0,1,-0.441052,-1.056280,1.327000,-0.450551,-0.309526,-0.368373,...,0.256375,-0.115861,0.381883,0.432567,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,2,-1.069696,2.0,0,-0.974273,-1.241098,0.136718,-1.344752,-1.005276,-1.227831,...,-1.137520,-0.855145,0.732664,-1.150309,0,0,0,0,0,0
168,1,0.831456,2.0,0,0.447649,0.607080,0.367096,0.694914,1.023995,0.231760,...,-1.806589,-0.458717,-0.600304,-1.424268,0,0,0,1,0,0
929,1,1.284112,2.0,1,0.625389,-0.224600,-0.784790,-0.199288,-0.251547,-0.058659,...,-0.329061,1.201991,-1.021241,-0.328431,1,1,0,1,0,0
2192,2,-0.707572,1.0,0,-0.441052,-0.363213,0.251907,0.827936,0.096328,1.078623,...,2.681752,0.259137,-1.091397,3.415680,1,0,0,0,0,0


### save to csv

In [23]:
train_df.to_csv('train_standard.csv', index=False)
test_df.to_csv('test_standard.csv', index=False)

In [24]:
train_df['target'].value_counts()

target
0    1815
1     160
Name: count, dtype: int64